In [1]:
%matplotlib notebook

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gmaps

In [80]:
#Read in each csv file 
fire_data_1 = pd.read_csv("Data/us_fires_1.csv", low_memory=False, encoding='latin-1')
fire_data_2 = pd.read_csv("Data/us_fires_2.csv", low_memory=False, encoding='latin-1')
fire_data_3 = pd.read_csv("Data/us_fires_3.csv", low_memory=False, encoding='latin-1')
fire_data_4 = pd.read_csv("Data/us_fires_4.csv", low_memory=False, encoding='latin-1')
fire_data_5 = pd.read_csv("Data/us_fires_5.csv", low_memory=False, encoding='latin-1')
fire_data_6 = pd.read_csv("Data/us_fires_6.csv", low_memory=False, encoding='latin-1')
fire_data_7 = pd.read_csv("Data/us_fires_7.csv", low_memory=False, encoding='latin-1')

#Combine the data frames into one 
all_data = pd.concat([fire_data_1, fire_data_2, fire_data_3, fire_data_4, 
                      fire_data_5, fire_data_6, fire_data_7], axis=0)

#Drop unnecessary columns 
cleaned_df = all_data.drop(['objectid', 'fod_id', 'fpa_id', 'source_system_type', 'source_system', 'nwcg_reporting_agency',
              'nwcg_reporting_unit_id', 'nwcg_reporting_unit_name', 'source_reporting_unit', 'owner_code', 
               'owner_descr', 'fips_code', 'fips_name', 'local_fire_report_id', 'local_incident_id',
              'fire_code', 'ics_209_incident_number', 'ics_209_name', 'mtbs_id', 'complex_name',
              'mtbs_fire_name', 'discovery_doy', 'discovery_time', 'stat_cause_code', 'cont_doy',
              'cont_time', 'fire_name', 'source_reporting_unit_name', 'cont_date', 'discovery_date','county'], axis=1)

#Rename columns 
cleaned_df.columns = ['Year', 'Cause', 'Size', 'Size Class','Latitude', 'Longitude', 'State']

In [86]:
#Rename columns 
cleaned_df.columns = ['Year', 'Cause', 'Size', 'Size Class','Latitude', 'Longitude', 'State']

#Sort data by state 
df = cleaned_df.sort_values('State', ascending=True)
df

,Year,Cause,Size,Size Class,Latitude,Longitude,State
162099,1994,Miscellaneous,0.5,B,64.883331,-147.966659,AK
210664,1993,Debris Burning,0.5,B,65.566300,-144.919100,AK
161788,1993,Miscellaneous,0.1,A,61.966667,-149.433334,AK
210663,1993,Lightning,4470.0,F,65.832900,-147.402500,AK
210662,1993,Lightning,5530.0,G,64.199700,-141.085600,AK
210661,1993,Lightning,7200.0,G,64.766200,-151.269100,AK
161789,1993,Lightning,15.0,C,64.666664,-145.733337,AK
210660,1993,Campfire,3.0,B,63.366300,-142.568900,AK
210659,1993,Lightning,5810.0,G,67.166300,-147.819400,AK
210658,1993,Lightning,30.0,C,66.383000,-143.419200,AK
